In [19]:
import quantstats as qs
import pandas_datareader as pdr
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [20]:
# Jupyter Notebook에서 보기 편하게 하기 위한 설정
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', None)

start_day = datetime(2008,1,1) # 시작일
end_day = datetime(2021,4,6) # 종료일

# RU : Risky Universe
# CU : Cash Universe
# BU : Benchmark Universe
RU = ['SPY','IWM','QQQ','VGK','EWJ','VWO','VNQ','GSG','GLD','TLT','HYG','LQD'] 
CU = ['SHV','IEF','UST','BND']
BU = ['^GSPC','^IXIC','^KS11','^KQ11'] # S&P500 지수, 나스닥 지수, 코스피 지수, 코스닥 지수

# Yahoo Finance에서 데이터 가져오는 함수
def get_price_data(RU, CU, BU):
    df_RCU = pd.DataFrame(columns=RU+CU)
    df_BU = pd.DataFrame(columns=BU)
    
    for ticker in RU + CU:
        df_RCU[ticker] = pdr.get_data_yahoo(ticker, start_day - timedelta(days=365), end_day)['Adj Close']  
    
    for ticker in BU:
        df_BU[ticker] = pdr.get_data_yahoo(ticker, start_day - timedelta(days=365), end_day)['Adj Close']  
     
    return df_RCU, df_BU

In [21]:
df_RCU, df_BU = get_price_data(RU, CU, BU)
df_RCU.head(5)

,SPY,IWM,QQQ,VGK,EWJ,VWO,VNQ,GSG,GLD,TLT,HYG,LQD,SHV,IEF,UST,BND
Date,,,,,,,,,,,,,,,,
2007-01-03,104.7521,63.6363,38.0272,38.6540,45.0964,26.3187,40.7015,38.4000,62.2800,56.5980,NaN,58.6263,NaN,57.5975,NaN,NaN
2007-01-04,104.9744,63.8073,38.7484,38.5807,45.5727,26.0316,40.7543,37.1500,61.6500,56.9411,NaN,59.0669,NaN,57.8065,NaN,NaN
2007-01-05,104.1371,62.4720,38.5637,38.0164,44.3977,25.3052,39.9996,37.4800,60.1700,56.6933,NaN,59.0284,NaN,57.6184,NaN,NaN
2007-01-08,104.6188,62.2278,38.5901,37.9599,44.6517,25.5822,40.0577,37.4000,60.4800,56.7950,NaN,59.0118,NaN,57.6602,NaN,NaN
2007-01-09,104.5299,62.8140,38.7836,37.8189,44.9376,24.9707,40.5643,37.2200,60.8500,56.7950,NaN,59.0228,NaN,57.6462,NaN,NaN


In [22]:
# 모멘텀 지수 계산 함수
def get_momentum(x):
    temp_list = [0 for i in range(len(x.index))]
    momentum = pd.Series(temp_list, index=x.index)

    try:
        before1 = df_RCU[x.name-timedelta(days=35):x.name-timedelta(days=30)].iloc[-1][RU+CU]
        before3 = df_RCU[x.name-timedelta(days=95):x.name-timedelta(days=90)].iloc[-1][RU+CU]        
        before6 = df_RCU[x.name-timedelta(days=185):x.name-timedelta(days=180)].iloc[-1][RU+CU]        
        before12 = df_RCU[x.name-timedelta(days=370):x.name-timedelta(days=365)].iloc[-1][RU+CU]

        momentum = 12 * (x / before1 - 1) + 4 * (x / before3 - 1) + 2 * (x / before6 - 1) + (x / before12 - 1)
    except:
        pass

    return momentum

In [23]:

# 각 자산별 모멘텀 지수 계산
mom_col_list = [col+'_M' for col in df_RCU[RU+CU].columns]
df_RCU[mom_col_list] = df_RCU[RU+CU].apply(lambda x: get_momentum(x), axis=1)
df_RCU[mom_col_list]

,SPY_M,IWM_M,QQQ_M,VGK_M,EWJ_M,VWO_M,VNQ_M,GSG_M,GLD_M,TLT_M,HYG_M,LQD_M,SHV_M,IEF_M,UST_M,BND_M
Date,,,,,,,,,,,,,,,,
2007-01-03,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2007-01-04,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2007-01-05,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2007-01-08,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2007-01-09,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30,1.6454,2.2521,0.9231,1.5053,1.0069,0.9720,1.8041,1.4438,-0.9736,-1.5909,0.3791,-0.4172,-0.0018,-0.6869,-1.3658,-0.3370
2021-03-31,1.4566,2.1252,0.9415,1.3111,0.6572,0.7725,1.5739,1.6179,-0.6570,-1.5278,0.3443,-0.3776,-0.0020,-0.7020,-1.3814,-0.3250
2021-04-01,1.8493,2.7710,1.5539,1.6651,1.0243,1.1215,2.1703,1.9056,-0.5008,-1.2504,0.4369,-0.2312,-0.0012,-0.6462,-1.2881,-0.2687


In [24]:
df_RCU = df_RCU[start_day:end_day]

# 매월 말일 데이터만 추출(리밸런싱에 사용)
df_RCU = df_RCU.resample(rule='M').last()
df_RCU.head(5)

,SPY,IWM,QQQ,VGK,EWJ,VWO,VNQ,GSG,GLD,TLT,HYG,LQD,SHV,IEF,UST,BND,SPY_M,IWM_M,QQQ_M,VGK_M,EWJ_M,VWO_M,VNQ_M,GSG_M,GLD_M,TLT_M,HYG_M,LQD_M,SHV_M,IEF_M,UST_M,BND_M
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-01-31,103.6666,58.3160,39.8076,39.6487,40.8159,32.6630,33.6617,52.5000,91.4000,63.2875,39.9592,62.5082,101.5181,65.5182,NaN,50.8475,-1.1690,-1.4077,-2.1727,-1.6517,-1.1514,-1.2911,-0.7722,0.6802,3.0316,0.8011,0.0000,0.6023,0.2520,1.0220,0.0000,0.0000
2008-02-29,100.9876,56.6062,37.8847,39.5323,40.2062,33.6028,32.5908,58.4100,96.1800,62.9993,39.2122,62.0667,101.7118,66.3301,NaN,50.8931,-0.6755,-0.8110,-1.2730,-0.3971,-0.7585,0.7824,-1.0032,3.1918,2.8657,0.3373,NaN,0.1055,0.1701,0.7379,NaN,NaN
2008-03-31,100.0845,56.2293,38.5940,40.0505,39.6928,32.3670,34.7044,57.8600,90.4100,64.3468,39.5832,61.6961,102.0325,67.2173,NaN,51.0965,-0.7976,-0.9403,-0.6548,-0.3444,-0.8448,-0.8088,0.4171,1.1560,0.5559,0.7467,NaN,0.0676,0.1849,0.7806,NaN,NaN
2008-04-30,104.8548,58.7900,41.6748,41.6923,42.6128,35.0967,36.9183,62.6000,86.6500,62.7470,40.9277,62.4640,102.1205,65.5923,NaN,50.9198,0.4135,0.2725,0.9150,0.5213,0.8964,1.3965,1.0210,2.6348,-0.2368,-0.1480,0.5189,0.2376,0.1228,-0.0362,NaN,0.1079
2008-05-31,106.4398,61.4825,44.1465,41.9718,43.4471,35.7231,36.8404,67.9000,87.4500,61.0601,40.7197,61.0866,102.1353,64.4264,NaN,50.4137,0.2382,0.7466,1.3839,0.1854,0.4391,0.6487,0.4295,3.0425,0.3440,-0.3925,0.0928,-0.3103,0.0926,-0.1804,NaN,-0.0621


In [25]:
# DAA 전략 기준에 맞춰 자산 선택
def select_asset(x):
    asset = pd.Series([0,0,0,0], index=['ASSET1','PRICE1','ASSET2','PRICE2'])
    momentum1 = None
    momentum2 = None
    
    # DAA 전략
    # 카나리아 자산군이 모두 0이상이면, 공격 자산 중 상위 2개 모멘텀 자산 선정
    # 'SPY','IWM','QQQ','VGK','EWJ','VWO','VNQ','GSG','GLD','TLT','HYG','LQD'
    if x['VWO_M'] > 0 and x['BND_M'] > 0:
        momentum_sort = x[['SPY_M','IWM_M','QQQ_M','VGK_M','EWJ_M','VWO_M','VNQ_M','GSG_M','GLD_M','TLT_M','HYG_M','LQD_M']].sort_values(ascending=False)
        momentum1 = momentum_sort[0]
        momentum2 = momentum_sort[1]

        asset['ASSET1'] = x[x == momentum1].index[0][:3]
        asset['PRICE1'] = x[asset['ASSET1']] 
        asset['ASSET2'] = x[x == momentum2].index[0][:3]
        asset['PRICE2'] = x[asset['ASSET2']]
    
    # 카나리아 자산군 중 하나라도 0이하라면, 방어 자산 중 최고 모멘텀 자산 선정
    # 'SHV','IEF','UST'
    else :
        momentum1 = max(x['SHV_M'],x['IEF_M'],x['UST_M'])
        
        asset['ASSET1'] = x[x == momentum1].index[0][:3]
        asset['PRICE1'] = x[asset['ASSET1']] 
        asset['ASSET2'] = x[x == momentum1].index[0][:3]
        asset['PRICE2'] = x[asset['ASSET2']]        
    
    return asset

In [26]:
# 매월 선택할 자산과 가격
df_RCU[['ASSET1','PRICE1','ASSET2','PRICE2']] = df_RCU.apply(lambda x: select_asset(x), axis=1)
df_RCU[['ASSET1','PRICE1','ASSET2','PRICE2']].tail(5)

,ASSET1,PRICE1,ASSET2,PRICE2
Date,,,,
2020-12-31,IWM,193.8373,QQQ,311.8503
2021-01-31,SHV,110.4117,SHV,110.4117
2021-02-28,SHV,110.4117,SHV,110.4117
2021-03-31,SHV,110.4018,SHV,110.4018
2021-04-30,SHV,110.4217,SHV,110.4217


In [27]:
# 각 자산별 수익률 계산
profit_col_list = [col+'_P' for col in df_RCU[RU+CU].columns]
df_RCU[profit_col_list] = df_RCU[RU+CU].pct_change()
df_RCU[profit_col_list].tail(5)

,SPY_P,IWM_P,QQQ_P,VGK_P,EWJ_P,VWO_P,VNQ_P,GSG_P,GLD_P,TLT_P,HYG_P,LQD_P,SHV_P,IEF_P,UST_P,BND_P
Date,,,,,,,,,,,,,,,,
2020-12-31,0.0370,0.0865,0.0490,0.0568,0.0534,0.0600,0.0273,0.0584,0.0701,-0.0123,0.0196,0.0015,0.0002,-0.0024,-0.0062,0.0015
2021-01-31,-0.0102,0.0485,0.0026,-0.0090,-0.0084,0.0313,0.0004,0.0470,-0.0322,-0.0363,-0.0038,-0.0183,-0.0001,-0.0109,-0.0223,-0.0086
2021-02-28,0.0278,0.0620,-0.0013,0.0258,0.0184,0.0157,0.0343,0.1030,-0.0626,-0.0573,-0.0024,-0.0226,0.0000,-0.0236,-0.0473,-0.0155
2021-03-31,0.0454,0.0140,0.0172,0.0332,0.0044,-0.0071,0.0515,-0.0190,-0.0114,-0.0525,0.0122,-0.0148,-0.0001,-0.0239,-0.0474,-0.0127
2021-04-30,0.0247,0.0153,0.0366,0.0241,0.0058,0.0159,0.0247,0.0000,0.0204,0.0190,0.0043,0.0080,0.0002,0.0065,0.0124,0.0052


In [28]:
# 매월 수익률 & 누적 수익률 계산
df_RCU['PROFIT'] = 0
df_RCU['PROFIT_ACC'] = 0
df_RCU['LOG_PROFIT'] = 0
df_RCU['LOG_PROFIT_ACC'] = 0

for i in range(len(df_RCU)):
    profit = 0
    log_profit = 0
        
    if i != 0:
        profit = (df_RCU[df_RCU.iloc[i-1]['ASSET1'] + '_P'].iloc[i] + df_RCU[df_RCU.iloc[i-1]['ASSET2'] + '_P'].iloc[i]) / 2
        log_profit = math.log(profit+1)
    
    df_RCU.loc[df_RCU.index[i], 'PROFIT'] = profit
    df_RCU.loc[df_RCU.index[i], 'PROFIT_ACC'] = (1+df_RCU.loc[df_RCU.index[i-1], 'PROFIT_ACC'])*(1+profit)-1
    df_RCU.loc[df_RCU.index[i], 'LOG_PROFIT'] = log_profit
    df_RCU.loc[df_RCU.index[i], 'LOG_PROFIT_ACC'] = df_RCU.loc[df_RCU.index[i-1], 'LOG_PROFIT_ACC'] + log_profit
    
# 수익률에 100을 곱해서 백분율로 표기   
df_RCU[['PROFIT', 'PROFIT_ACC', 'LOG_PROFIT','LOG_PROFIT_ACC']] = df_RCU[['PROFIT', 'PROFIT_ACC', 'LOG_PROFIT','LOG_PROFIT_ACC']] * 100
df_RCU[profit_col_list] = df_RCU[profit_col_list] * 100    

df_RCU[['PROFIT','PROFIT_ACC','LOG_PROFIT','LOG_PROFIT_ACC']].tail(10)

,PROFIT,PROFIT_ACC,LOG_PROFIT,LOG_PROFIT_ACC
Date,,,,
2020-07-31,5.1330,412.4193,5.0056,163.3973
2020-08-31,5.3106,439.6317,5.1744,168.5717
2020-09-30,-4.7615,413.9371,-4.8786,163.6931
2020-10-31,-2.2250,402.5020,-2.2501,161.4429
2020-11-30,-0.0208,402.3976,-0.0208,161.4222
2020-12-31,7.1620,438.3794,6.9172,168.3393
2021-01-31,2.5534,452.1264,2.5213,170.8607
2021-02-28,0.0000,452.1264,0.0000,170.8607
2021-03-31,-0.0090,452.0765,-0.0090,170.8516


In [29]:
stock = qs.utils.download_returns('FB')
stock.tail(10)

In [30]:
qs.reports.metrics(stock, mode='basic')

                    Strategy
------------------  ----------
Start Period        2012-05-18
End Period          2022-06-09
Risk-Free Rate      0.0%
Time in Market      100.0%

Cumulative Return   381.27%
CAGR﹪              16.89%

Sharpe              0.6
Prob. Sharpe Ratio  96.96%
Sortino             0.88
Sortino/√2          0.62
Omega               1.12

Max Drawdown        -54.22%
Longest DD Days     532

Gain/Pain Ratio     0.12
Gain/Pain (1M)      0.68

Payoff Ratio        1.02
Profit Factor       1.12
Common Sense Ratio  1.13
CPC Index           0.6
Tail Ratio          1.0
Outlier Win Ratio   3.9
Outlier Loss Ratio  4.0

MTD                 -4.98%
3M                  -3.31%
6M                  -44.34%
YTD                 -45.3%
1Y                  -44.86%
3Y (ann.)           -2.1%
5Y (ann.)           4.04%
10Y (ann.)          21.11%
All-time (ann.)     16.89%

Avg. Drawdown       -5.37%
Avg. Drawdown Days  30
Recovery Factor     7.03
Ulcer Index         0.18
Serenity Index      1.5

In [31]:
qs.reports.html(stock, "FB", output='./FB.html')